In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import shapely.wkt
from shapely.geometry import Point, Polygon, MultiPolygon

In [ ]:
year = '2006'
file311 = "/content/gdrive/MyDrive/Dataset/311_06.csv"
fileCrime = "/content/gdrive/MyDrive/Dataset/CrimeData.csv"
fileRegions = "/content/gdrive/MyDrive/Dataset/Regions.csv"


In [ ]:
year = '2006'
file311 = "RawData/311_06.csv"
fileCrime = "RawData/CrimeData.csv"
fileRegions = "RawData/Regions.csv"

In [ ]:
print(year)

2006


In [ ]:
print(file311)

RawData/311_06.csv


In [ ]:
print("RawData/311_06.csv")

RawData/311_06.csv


In [ ]:
data311=pd.read_csv('/content/gdrive/MyDrive/Dataset/311_06.csv')

In [ ]:
datacrime=pd.read_csv('/content/gdrive/MyDrive/Dataset/CrimeData.csv')

In [ ]:
dataregions=pd.read_csv('/content/gdrive/MyDrive/Dataset/Regions.csv')

In [ ]:
data311 = pd.read_csv('/content/gdrive/MyDrive/Dataset/311_06.csv',low_memory=False)

In [ ]:
relevantColumns311 = ['Created Date','Latitude','Longitude','Complaint Type']
relevantComplaints311 = ['Blocked Driveway','Building/Use','Noise','Safety']
finalColumns311 = ['Created Date','Complaint Type','Precincts']
data311 = data311[relevantColumns311]
data311 = data311.loc[(data311['Complaint Type'] == 'Blocked Driveway') | (data311['Complaint Type'] == 'Building/Use') | (data311['Complaint Type'] == 'Noise') | (data311['Complaint Type'] == 'Safety')]
print('Anomalies preprocessed!!!')

In [ ]:
print(relevantColumns311)

In [ ]:
print(data311)

In [ ]:
crimeData = pd.read_csv('/content/gdrive/MyDrive/Dataset/CrimeData.csv', low_memory=False)

In [ ]:
relevantColumnsCR = ['CMPLNT_TO_DT','CMPLNT_TO_TM','CMPLNT_FR_DT','RPT_DT','Lat_Lon','Latitude','Longitude','OFNS_DESC']
finalColumnsCR = ['Date','OFNS_DESC','CMPLNT_TO_TM','Longitude','Latitude','Precincts']
relevantCrimesCR = ['ROBBERY', 'BURGLARY', 'FELONY ASSAULT','GRAND LARCENY']
locations = pd.read_csv('/content/gdrive/MyDrive/Dataset/Regions.csv', low_memory=False)
crime2006 = crimeData.loc[crimeData['RPT_DT'].str.endswith(year)]
crime2006 = crime2006[relevantColumnsCR]
crime2006 = crime2006.loc[(crime2006['OFNS_DESC'] == 'ROBBERY') | (crime2006['OFNS_DESC'] == 'BURGLARY') | (crime2006['OFNS_DESC'] == 'FELONY ASSAULT') | (crime2006['OFNS_DESC'] == 'GRAND LARCENY')]
print('Crime Preprocessed!!!')

In [ ]:
print(crime2006)

In [ ]:
locations = pd.read_csv("/content/gdrive/MyDrive/Dataset/Regions.csv", low_memory=False)
precincts = {}
for index, row in locations.iterrows():
  precincts[row['Precinct']] = shapely.wkt.loads(row['the_geom'])

print('Locations loaded!!!')

In [ ]:
#In dataset 311
pos = 0
prec = np.ndarray((data311.shape[0],))

for index,row in data311.iterrows():

  poo = Point(row['Longitude'],row['Latitude'])  
  for key,val in precincts.items():
    if poo.within(val):
      prec[pos] = key
      break

  pos=pos+1
  if(pos%1000 == 0):
    print ("processed "+str(pos)+" records!!")

# Adds the precincts information to the dataset
data311['Precincts'] = prec.astype(int)
print("Done!!")

# Extract the final columns needed for training the model
data311['Precincts'] = data311['Precincts'].astype(np.int64)
data311 = data311[data311['Precincts'] >= 0]
data311 = data311[finalColumns311]

print('Anomaly Preprocessing complete!!!')

processed 1000 records!!
processed 2000 records!!
processed 3000 records!!
processed 4000 records!!
processed 5000 records!!
processed 6000 records!!
processed 7000 records!!
processed 8000 records!!
processed 9000 records!!
processed 10000 records!!
processed 11000 records!!
processed 12000 records!!
processed 13000 records!!
processed 14000 records!!
processed 15000 records!!
processed 16000 records!!
processed 17000 records!!
processed 18000 records!!
processed 19000 records!!
processed 20000 records!!
processed 21000 records!!
processed 22000 records!!
processed 23000 records!!
processed 24000 records!!
processed 25000 records!!
processed 26000 records!!
processed 27000 records!!
processed 28000 records!!
processed 29000 records!!
processed 30000 records!!
processed 31000 records!!
processed 32000 records!!
processed 33000 records!!
processed 34000 records!!
processed 35000 records!!
processed 36000 records!!
processed 37000 records!!
processed 38000 records!!
processed 39000 recor

In [ ]:
pos = 0
prec = np.ndarray((crime2006.shape[0],))
date = np.ndarray((crime2006.shape[0],)).astype(str)

for index,row in crime2006.iterrows():
  
  if not(str(row['CMPLNT_FR_DT']) == 'nan'):
    date[pos] = row['CMPLNT_FR_DT']
  elif not(str(row['CMPLNT_TO_DT']) == 'nan'):
    date[pos] = row['CMPLNT_TO_DT']
  else:
    date[pos] = row['RPT_DT']
  
  poo = Point(row['Longitude'],row['Latitude'])  
  for key,val in precincts.items():
    if poo.within(val):
      prec[pos] = key
      break

  pos=pos+1
  if(pos%1000 == 0):
    print ("processed "+str(pos)+" records!!")

# Adds the precincts information to the dataset
# Adds the dates information to the dataset

crime2006['Precincts'] = prec.astype(int)
crime2006['Date'] = date
crime2006 = crime2006[crime2006['Precincts'] >= 0]

print("Done!!")

# Extract the final columns needed for training the model
crime2006 = crime2006[finalColumnsCR]
crime2006.sort_values(['Precincts','Date','OFNS_DESC'],ascending=[True,True,True],inplace=True)
crime2006 = crime2006[crime2006['Date'].str.contains(year)]
crime2006['Precincts'] = crime2006['Precincts'].astype(np.int64)

print('Crime Preprocessing complete!!!')

processed 1000 records!!
processed 2000 records!!
processed 3000 records!!
processed 4000 records!!
processed 5000 records!!
processed 6000 records!!
processed 7000 records!!
processed 8000 records!!
processed 9000 records!!
processed 10000 records!!
processed 11000 records!!
processed 12000 records!!
processed 13000 records!!
processed 14000 records!!
processed 15000 records!!
processed 16000 records!!
processed 17000 records!!
processed 18000 records!!
processed 19000 records!!
processed 20000 records!!
processed 21000 records!!
processed 22000 records!!
processed 23000 records!!
processed 24000 records!!
processed 25000 records!!
processed 26000 records!!
processed 27000 records!!
processed 28000 records!!
processed 29000 records!!
processed 30000 records!!
processed 31000 records!!
processed 32000 records!!
processed 33000 records!!
processed 34000 records!!
processed 35000 records!!
processed 36000 records!!
processed 37000 records!!
processed 38000 records!!
processed 39000 recor

In [ ]:
daterange = pd.date_range(start='01/01/'+year,end='12/31/'+year)
daterange = daterange.strftime('%m/%d/%Y')
daterange = daterange.tolist()

inv_dict = {}
for i in range(len(daterange)):
  inv_dict[daterange[i]] = i

inv_complaints = {}
for i in range(len(relevantComplaints311)):
  inv_complaints[relevantComplaints311[i]] = i

n_precincts = data311['Precincts'].nunique()
uniq_precincts = data311['Precincts'].unique()
inv_prec = {}
for i in range(n_precincts):
  inv_prec[uniq_precincts[i]] = i

inv_crimes = {}
for i in range(len(relevantCrimesCR)):
  inv_crimes[relevantCrimesCR[i]] = i

In [ ]:
matricesCR = [np.zeros((n_precincts,4),dtype=np.int64) for x in range(365)]
exceptions = 0

for idx, row in crime2006.iterrows():
  try:
    id1 = inv_prec[row['Precincts']]
    id2 = inv_dict[row['Date']]
    id3 = inv_crimes[row['OFNS_DESC']]
    matricesCR[id2][id1][id3] = matricesCR[id2][id1][id3] + 1
  except:
    print("Exception!!!")
    print("Precincts",id1)
    print("Date",id2)
    print("Offense",id3)
    exceptions = exceptions + 1

print ('Created crime matrices!!!')

# Generating the 311 complaints records matrices
matrices311 = [np.zeros((n_precincts,4),dtype=np.int64) for x in range(365)]
exceptions = 0

for idx, row in data311.iterrows():
  try:
    id1 = inv_prec[row['Precincts']]
    id2 = inv_dict[str(row['Created Date']).split()[0]]
    id3 = inv_complaints[row['Complaint Type']]
    matrices311[id2][id1][id3] = matrices311[id2][id1][id3] + 1
  except:
    print("Exception!!!")
    print("Precincts",id1)
    print("Date",id2)
    print("Offense",id3)
    exceptions = exceptions + 1

print('Created anomaly matrices!!!')

Created crime matrices!!!
Created anomaly matrices!!!


In [ ]:
import pickle


# Dump the matrices using pickle, to a file
output311 = 'matrices311'
outputCrime = 'matricesCR'

file311 = open(output311,'wb')
pickle.dump(matrices311,file311)

fileCR = open(outputCrime,'wb')
pickle.dump(matricesCR,fileCR)


In [ ]:
print(output311)

matrices311


In [ ]:
print(matrices311)

[array([[ 2,  0,  0,  0],
       [ 3,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  1,  1,  0],
       [ 1,  0,  1,  0],
       [ 1,  1,  0,  0],
       [ 8,  1,  1,  0],
       [ 1,  0,  1,  0],
       [ 3,  1,  0,  0],
       [ 0,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 2,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 4,  0,  1,  0],
       [ 5,  0,  2,  0],
       [ 0,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 4,  0,  2,  0],
       [ 2,  1,  0,  0],
       [ 1,  0,  0,  0],
       [ 4,  0,  0,  0],
       [ 0,  0,  1,  0],
       [10,  0,  0,  0],
       [ 5,  0,  1,  0],
       [ 0,  0,  1,  0],
       [ 0,  0,  1,  0],
       [ 5,  0,  0,  0],
       [ 5,  0,  2,  0],
       [ 3,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 8,  1,  0,  0],
       [13,  0,  0,  0],
       [ 5,  0,  1,  0],
       [ 0,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 3,  0,  0,  0],
       [ 1,  1,  0,  0],
       [ 0,  0,  1,  0],

In [ ]:
print(matricesCR)

NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
import shapely.wkt
from shapely.geometry import Point, Polygon, MultiPolygon
import re

# Reads the point-of-interest dataset from New York City
# Important locations, classified as following, with the geographical data and date of creation/updation to records

# 1 Residential
# 2 Education Facility
# 3 Cultural Facility
# 4 Recreational Facility
# 5 Social Services
# 6 Transportation Facility
# 7 Commercial
# 8 Government Facility (non public safety)
# 9 Religious Institution
# 10 Health Services
# 11 Public Safety
# 12 Water
# 13 Miscellaneous

filePOI = '/content/gdrive/MyDrive/Dataset/Point_Of_Interest.csv'
fileRegions ='/content/gdrive/MyDrive/Dataset/Regions.csv'

poi = pd.read_csv(filePOI,low_memory=False)
locations = pd.read_csv(fileRegions, low_memory=False)

relevantColumns = ['the_geom','CREATED','FACILITY_T']
finalColumns = ['Precincts','FACILITY_T']
poi = poi[relevantColumns]

# Reading geographical divisions of precincts to localize the POIs to their respective regions
precincts = {}
for index, row in locations.iterrows():
  precincts[row['Precinct']] = shapely.wkt.loads(row['the_geom'])

In [ ]:
lat = []
longt = []
date = []

for idx,row in poi.iterrows():
  location = re.split(r' |\)|\(' ,row['the_geom'])
  year = int(row['CREATED'].split()[0].split('/')[-1])
  lat = lat + [location[-2]]
  longt = longt + [location[2]]
  date = date + [year]

poi['Year'] = date
poi['Latitude'] = lat
poi['Longitude'] = longt


# Assigning the precinct number to a location in which the property is
pos = 0
prec = np.ndarray((poi.shape[0],))

for index,row in poi.iterrows():

  poo = Point(float(row['Longitude']),float(row['Latitude']))  
  for key,val in precincts.items():
    if poo.within(val):
      prec[pos] = key
      break

  pos=pos+1
  if(pos%1000 == 0):
    print ("processed "+str(pos)+" records!!")
    
poi['Precincts'] = prec.astype(int)
print("Done!!")

poi['Precincts'] = poi['Precincts'].astype(np.int64)
poi = poi[poi['Precincts'] >= 0]


# Inverted dictionaries for easy conversion of the above dataframe to a matrix that is usable later in the neural architecture

# For categories of the locality, listed above
inv_categories = {}
n_categories = poi['FACILITY_T'].nunique()
uniq_categories = poi['FACILITY_T'].unique()
for i in range(n_categories):
  inv_categories[uniq_categories[i]] = i

# For geographical division of precincts
n_precincts = poi['Precincts'].nunique()
uniq_precincts = poi['Precincts'].unique()
inv_prec = {}
for i in range(n_precincts):
  inv_prec[uniq_precincts[i]] = i

poi = poi[poi['Year'] <= 2008]
poi = poi[finalColumns]

In [ ]:
matrices = np.zeros((n_precincts,n_categories),dtype=np.int64)
exceptions = 0

for idx, row in poi.iterrows():
  try:
    id1 = inv_prec[row['Precincts']]
    id2 = inv_categories[row['FACILITY_T']]
    matrices[id1][id2]= matrices[id1][id2] + 1
  except:
    print("Exception!!!")
    print("Precincts",id1)
    print("FACILITY_T",id2)
    exceptions = exceptions + 1

In [ ]:
import pickle

outputPOI = 'poiMatrices'

# Pickle dump this file
file = open(outputPOI,'wb')
pickle.dump(matrices,file)

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==1.14

Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
  Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203
     |████████████████████████████████| 109.3 MB 42 kB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 488 kB 54.8 MB/s 
     |████████████████████████████████| 3.1 MB 39.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tens

In [ ]:
# LSTM and RNN code derived from the following github repo: https://github.com/TobiasLee/Text-Classification

import numpy as np
import pandas as pd
import time
import pickle

from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
import tensorflow as tf
tf.disable_v2_behavior()

from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
input311File = 'matrices311'
inputCrimeFile = 'matricesCR'

with open(inputCrimeFile, 'rb') as pickle_file:
    anomaly = pickle.load(pickle_file)

with open(inputCrimeFile,'rb') as pickle_file:
    content = pickle.load(pickle_file)

FileNotFoundError: ignored

In [ ]:
dat = []
dat2 = []
for i in range(len(content)):
  a = []
  b = []
  for j in range(77):
    a.extend(content[i][j+1])
    b.extend(anomaly[i][j+1])
    # print(a)
  dat.append(a)
  dat2.append(b)

inp = np.array(dat)
inp1 = np.where(inp>0,1,0)
inp = np.array(dat)
inp1 = np.where(inp>0,1,0)
inpA = np.array(dat2)
inpA = np.array(dat2)

NameError: ignored